In [ ]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
import urllib.request
import hashlib
import Crypto
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA256, SHA, SHA512
from Crypto import Random
import base64
from copy import deepcopy

In [ ]:
HashPointer = namedtuple('HashPointer', ['hash', 'pointer'])
Transaction = namedtuple('Transaction', ['payer', 'payee', 'amount'])
Puzzle = namedtuple('Puzzle', ['difficulty', 'denominator', 'numerator'])
User = namedtuple('User', ['secret_key', 'public_key']) 

class Block:
    def __init__(self, transaction, signature, prev, nonce):
        self.transaction = transaction
        self.signature = signature
        self.prev = prev
        self.nonce = nonce
    
    def __repr__(self):
        return f'\nBlock(\n transaction: {self.transaction},\n signature: {self.signature},\n nonce: {self.nonce},\n prev: {self.prev})' 
    
class Blockchain:
    def __init__(self):
        self.users = {}
        self.length = 0
        self.blockchain = None
    
    def get_puzzle(self):
        return self.puzzle
        
    def build_genesis(self, transaction, signature):
        if self.length == 0:
            isValid, errors = self.process_transaction(transaction, signature)

            if isValid:
                self.blockchain = Block(transaction, signature, None, None)
                self.puzzle = self.new_puzzle()
                self.length = 1
                return True
            else:
                print(errors)
                return False
        else:
            print('Genesis already exists.')
            return False
        
    def add_transaction(self, transaction, signature, nonce, isPuzzle=False):
        isValid, errors = self.process_transaction(transaction, signature)
        
        if isValid:
            prev_hash = self.hash_object(self.blockchain)
            prev = HashPointer(prev_hash, self.blockchain)
            
            self.blockchain = Block(transaction, signature, prev, nonce)
            self.length  = self.length + 1
            if isPuzzle:
                self.broadcast('A new block has been added!')
            else:
                self.broadcast('A new transaction has been added!')
            return True
        else:
            print(errors)
            return False
        
    def new_user(self):  
        length=1024  
        secret_key = RSA.generate(length, Random.new().read)  
        public_key = secret_key.publickey()
        user = User(secret_key, public_key)
        return user
        
    def sign(self, sk, transaction):
        hashed_transaction = self.hash_object(transaction)
        return base64.b64encode(str((sk.sign(hashed_transaction,''))[0]).encode())
    
    def verify_signature(self, pk, sig, transaction):
        if pk is None:
            return True
        else:
            return pk.verify(self.hash_object(transaction),(int(base64.b64decode(sig)),))
    
    def process_transaction(self, transaction, signature):
        users = self.users
        payer = transaction.payer
        payer_hash = self.hash_object(payer)
        payee = transaction.payee
        payee_hash = self.hash_object(payee)
        amount = transaction.amount
        isValid = True
        error = ''
        
        if not self.verify_signature(payer, signature, transaction):
            isValid = False
            error += '\n Invalid signature.'
        
        if payer is None and payee is not None:
            if payee_hash not in users.keys():
                users[payee_hash] = amount
            else:
                isValid = False
                error += '\n This user already exists in the system.'
        elif payer is not None and payee is not None:
            if payee_hash in users.keys() and payer_hash in users.keys():
                payer_balance = users[payer_hash]
                payee_balance = users[payee_hash]
                
                if payer_balance - amount < 0:
                    isValid = False
                    error += '\n Payer does not have enough currency. '
                else:
                    users[payer_hash] = payer_balance - amount
                    users[payee_hash] = payee_balance + amount
                    
            else:
                if payee_hash not in users.keys():
                    isValid = False
                    error += '\n Payee does not exist. '
                if payer_hash not in users.keys():
                    isValid = False
                    error += '\n Payer does not exist. '
                
        self.users = users
        return isValid, error
    
    def new_puzzle(self):
        try:
            self.puzzle
        except AttributeError:
            denominator = 1  
        else:
            denominator = self.puzzle.denominator     
        
        hash_string = hashlib.sha256(bytes('hello', encoding='utf-8')).digest()
        denominator = np.random.randint(denominator, denominator*17)
        numerator = 2**(len(hash_string) * 8)
        difficulty = int(numerator/denominator)
        return Puzzle(difficulty, denominator, numerator)
    
    def verify_puzzle_solution(self, final_hash):
        difficulty = self.puzzle.difficulty
        
        if int(final_hash, 16) <= difficulty:
            return True
        return False
    
    def get_puzzle_hash(self, transaction, signature, nonce):
        isValid, errors = self.process_transaction(transaction, signature)
        
        if isValid:
            prev_hash = hashlib.sha256(bytes(str(self.blockchain), encoding='utf-8')).hexdigest()
            prev = HashPointer(prev_hash, self.blockchain)
            new_block = Block(transaction, signature, prev, nonce)
            block_hash = hashlib.sha256(bytes(str(new_block), encoding='utf-8')).hexdigest()
            return block_hash
        else:
            print(errors)
            return False
    
    def mine(self, transaction, signature):
        blockchain = self.blockchain
        nonce = 0
        finished = False
        
        while not finished:
            nonce = nonce + 1
            final_hash = self.get_puzzle_hash(transaction, signature, nonce)
            
            if self.verify_puzzle_solution(final_hash):
                self.add_transaction(transaction, signature, nonce, True)
                self.puzzle = self.new_puzzle()
                finished = True
                
    def check_blockchain(self, blockchain, expected_hash):
        block_to_check = blockchain

        while True:
            recomputed_hash = self.hash_object(block_to_check)

            if recomputed_hash != expected_hash:
                print('Blockchain failed to validate.')
                return False

            prev_pointer = block_to_check.prev

            if prev_pointer is None:
                return True
            else:
                block_to_check = prev_pointer.pointer
                expected_hash = prev_pointer.hash
    
    def hash_object(self, obj):
        return int.from_bytes(hashlib.sha256(bytes(str(obj), encoding='utf-8')).digest(), 'big')
    
    def broadcast(self, message):
        print('\n##################')
        print(message)
        print('##################\n')
        

In [ ]:
## Create transaction, create signature of that transaction, and use it to create the blockchain.
bc = Blockchain()

u1 = bc.new_user()

t1 = Transaction(None, u1.public_key, 1000)
s1 = bc.sign(u1.secret_key, t1)

assert bc.blockchain == None
assert bc.length == 0
print(bc.blockchain)
bc.build_genesis(t1, s1)
print(bc.blockchain)
assert bc.length == 1
assert bc.users[bc.hash_object(u1.public_key)] == 1000 

In [ ]:
## Add a transaction to the above blockchain
u2 = bc.new_user()
t2 = Transaction(None, u2.public_key, 3500)
s2 = bc.sign(u2.secret_key, t2)

bc.add_transaction(t2, s2, None)
assert bc.length == 2
assert bc.users[bc.hash_object(u2.public_key)] == 3500 
print(bc.blockchain)

In [ ]:
## Add a transaction that has one user pay another user
print(bc.users)
t3 = Transaction(u2.public_key, u1.public_key, 500)
s3 = bc.sign(u2.secret_key, t3)

bc.add_transaction(t3, s3, None)
assert bc.length == 3
assert bc.users[bc.hash_object(u1.public_key)] == 1500
assert bc.users[bc.hash_object(u2.public_key)] == 3000 
print(bc.users)
print(bc.blockchain)

In [ ]:
## Try to pay from an account that does not yet exist in the system
u3 = bc.new_user()
t4 = Transaction(u3.public_key, u2.public_key, 100)
s4 = bc.sign(u3.secret_key, t4)

bc.add_transaction(t4, s4, None)
assert bc.length == 3
assert bc.users[bc.hash_object(u2.public_key)] == 3000 

In [ ]:
## Try to pay to an account that does not yet exist in the system
u4 = bc.new_user()
t5 = Transaction(u2.public_key, u4.public_key, 100)
s5 = bc.sign(u2.secret_key, t5)

print(bc.users)
bc.add_transaction(t5, s5, None)
assert bc.length == 3
assert bc.users[bc.hash_object(u2.public_key)] == 3000 
print(bc.users)

In [ ]:
## Try mining with a user until completion
mt1 = Transaction(u1.public_key, None, 1)
ms1 = bc.sign(u1.secret_key, mt1)
bc.mine(mt1, ms1)

In [ ]:
# Test check blockchain
bc.check_blockchain(bc.blockchain, bc.hash_object(bc.blockchain))

In [ ]:
# Alter a transaction to test the failure of checking the blockchain
blockchain_copy = deepcopy(bc.blockchain)
blockchain_copy.transaction = Transaction('Joe', 'Bob', 20)
bc.check_blockchain(blockchain_copy, bc.hash_object(bc.blockchain))